<a href="https://colab.research.google.com/github/viniciusmmartins/PosFiap-AI-TechChallange-Fase3/blob/main/Tech_Challange_fase3_Gemma_4B_unsloth_bnb_4bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**negrito** Fine-tuning de LLM com dados médicos internos (✅)

    [X] Realizar o fine-tuning de um modelo LLM (como LLaMA, Falcon ou outro) utilizando:
        Protocolos médicos do hospital
        Exemplos de perguntas frequentes feitas por médicos
        Modelos de laudos, receitas e procedimentos internos
    Preparar os dados com técnicas de preprocessing, anonimização e curadoria
    Aulas:
        FASE 3: Fine-tuning e RAG para documentos
            Técnicas de Fine-tuning + RAG para consulta de dados (protocolos médicos do hospital + FAQ + Modelos)
            Tratamento de dataset https://pubmedqa.github.io/ + Mock (protocolos médicos do hospital + FAQ + Modelos)

 Criação de assistente médico com LangChain (✅)

    [X] Utilizar o LangChain para:
        Construir um pipeline que integre a LLM customizada
        Realizar consultas em base de dados estruturadas (como prontuários e registros)
        Contextualizar as respostas da LLM com informações atualizadas do paciente
    Aulas:
        FASE 3: LangChain na prática
            Iteração com LLMs
        FASE 3: LangGraph
            Iteração com LLMs usando workflows (nós)

 Segurança e validação (✅)

    [X] Definir os limites de atuação do assistente para evitar sugestões impróprias (exemplo: nunca prescrever diretamente, sem a validação humana)
    Implementar logging detalhado para rastreamento e auditoria
    Garantir explainability das respostas da LLM (exemplo: indicar a fonte da informação utilizada na resposta)
    Aulas:
        FASE 3: IA: Guia de Prompts
            Prompt engineering
        6IADT - Guardrails e validação
            Gravação de aula (Zoom)  Senha: &7woyXRX

 Organização do código (✅)

    Projeto modularizado em Python
    Instruções completas no README



In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

# Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


# Imports

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from google.colab import userdata

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


# Definindo o modelo

In [ ]:
#https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Gemma3_(4B).ipynb#scrollTo=-Xbb0cuLzwgf
max_seq_length = 2048
dtype = None
load_in_4bit = True
model_id = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_id,
    max_seq_length,
    load_in_4bit=load_in_4bit, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)


==((====))==  Unsloth 2025.12.5: Fast Gemma3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


# Formatando o dataset para fine-tunning

In [ ]:
import json
from datasets import Dataset

from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

def convert_to_conversation(sample, index):
  convos = []
  num_samples_in_batch = len(sample["QUESTION"])

  for i in range(num_samples_in_batch):
    contexts_single = sample["CONTEXTS"][i] if sample["CONTEXTS"][i] else []
    formatted_contexts = " ".join(contexts_single)
    labels_single = sample["LABELS"][i] if sample["LABELS"][i] else []
    formatted_labels = " ".join(labels_single)

    convos.append([
            {
              'content': f"""
              ## CONTEXT
              {formatted_contexts}


              ## QUESTION
              {sample["QUESTION"][i]}

              """,
              'role': 'user'
            },
            {
              'content': f"""
              ## ANSWER
              {sample["final_decision"][i]}. {sample["LONG_ANSWER"][i]}

              ## METADATA

              ### YEAR
              {sample["YEAR"][i]}

              ### LABEL
              {formatted_labels}
              """,
              'role': 'assistant'
            }
          ])

  texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
  return { "text" : texts, }


OUTPUT_PATH_DATASET = "/content/drive/MyDrive/AI/tech challenge/ori_pqal.json"

# Load the JSON file manually as it's a dictionary of dictionaries
with open(OUTPUT_PATH_DATASET, "r") as f:
    raw_data = json.load(f)

# Convert the dictionary of records into a list of records
dataset_list = list(raw_data.values())

# Create a Dataset object from the list
dataset = Dataset.from_list(dataset_list)

# Now, apply the map function to the correctly loaded dataset
dataset = dataset.map(convert_to_conversation, batched=True, with_indices=True)

print('RESPONSE FINAL',  dataset[0]['text'])


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

RESPONSE FINAL <start_of_turn>user
## CONTEXT
              Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). Window stage leaves were 

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1000 [00:00<?, ? examples/s]

Melhoria do fine tunning, reduzindo a perda nas respostas das roles users.

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=6):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 16,394,240 of 4,316,473,712 (0.38% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,5.680400
2,5.510900
3,5.453200
4,4.392800
5,3.960300
6,2.873600
7,2.651400
8,2.312400
9,2.358000
10,2.020100


# Salvar modelo

In [ ]:
#local
# model.save_pretrained("/content/drive/MyDrive/AI/tech challenge/modelos/lora_model")
# tokenizer.save_pretrained("/content/drive/MyDrive/AI/tech challenge/modelos/lora_model")

#hugginface
model.push_to_hub("gerson-analista/gemma-3-4b-lora_model", token = userdata.get('HF_TOKEN')) # Online saving
tokenizer.push_to_hub("gerson-analista/gemma-3-4b-lora_model", token = userdata.get('HF_TOKEN')) # Online saving

README.md:   0%|          | 0.00/606 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   0%|          | 17.0kB / 65.7MB            

Saved model to https://huggingface.co/gerson-analista/gemma-3-4b-lora_model


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpuyuvv048/tokenizer.json: 100%|##########| 33.4MB / 33.4MB            

  ...puyuvv048/tokenizer.model: 100%|##########| 4.69MB / 4.69MB            

No files have been modified since last commit. Skipping to prevent empty commit.


## Vamos carregar do huggingface, caso deseje mudar há a necessidade de ajustar o código acima

In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name = "/content/drive/MyDrive/AI/tech challenge/modelos/lora_model", # YOUR MODEL YOU USED FOR TRAINING
    model_name = "gerson-analista/gemma-3-4b-lora_model", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.12.5: Fast Gemma3 patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


adapter_model.safetensors:   0%|          | 0.00/65.7M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma3ForConditionalGeneration(
      (model): Gemma3Model(
        (vision_tower): SiglipVisionModel(
          (vision_model): SiglipVisionTransformer(
            (embeddings): SiglipVisionEmbeddings(
              (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
              (position_embedding): Embedding(4096, 1152)
            )
            (encoder): SiglipEncoder(
              (layers): ModuleList(
                (0-26): 27 x SiglipEncoderLayer(
                  (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
                  (self_attn): SiglipAttention(
                    (k_proj): lora.Linear(
                      (base_layer): Linear(in_features=1152, out_features=1152, bias=True)
                      (lora_dropout): ModuleDict(
                        (default): Identity()
                      )
                      (lora_A): ModuleDict(
  

## Testando o modelo pré-treinado


In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "My patient has high blood pressure, does it increases the chance of heart attack?",}]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = True,
    return_tensors = "pt",
    return_dict = True,
)

from transformers import TextStreamer
_ = model.generate(
    **inputs.to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

you’re asking a very important and crucial question about your patient, I’m glad you’re asking this about how high blood pressure will affect them.

yes, that is a good and relevant question to ask. you will want to provide this information for your patient so that they will understand and have a chance
